# <h2><b> Đồ án thực hành </b></h2>
<h1 style='color:red;text-align:center;font-size:35px'><b>Chủ đề: Dự đoán giá rượu vang</b></h1>

# <h3>Các thành viên trong nhóm</h3>
<li> Phan Xuân Hoài - 20120481 </li>
<li> Lê Kinh Luân - 1612355 </li>
<li> Nguyễn Minh Nghĩa - 20120534</li>
<li> Phạm Quốc Hưng - 19120522</li>

# <h1 style='color:blue;text-align:center;font-size:30px'><b>Phần I - C.Đặt các câu hỏi có ý nghĩa cần trả lời</b></h1>

## Import những thư viện cần thiết

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns
sns.set_theme()

# YOUR CODE HERE (OPTION)
# Nếu cần các thư viện khác thì bạn có thể import ở đây
import os
import re
import statistics
import seaborn as sns
sns.set(rc={'axes.facecolor':'lightblue', 'figure.facecolor':'lightgray'})

In [ ]:
import sys
sys.executable

In [ ]:
cd champagne/champagne

### Đọc file 'champagnes_info_cleaned.csv' vào dataframe df

In [ ]:
df = pd.read_csv('./dataset/champagne_info_cleaned.csv')
df.head(20)

In [ ]:
df.info()

<h2 style='color:#8edc55'>Câu 1. Giữa màu sắc, thương hiệu có mối tương quan như thế nào đối với giá rượu?  </h2>

**Lợi ích khi trả lời được câu hỏi:** 
+ Với người mua, họ có thể tham khảo để xác định được giá của một chai rượu vang họ muốn mua có đang là hợp lý, tránh bị lừa.
+ Với người bán, họ có thể tham khảo để đưa ra một mức giá bán hợp lý cho sản phẩm

**Nguồn cảm hứng của câu hỏi:** tự nghĩ

---

<h3 style='color:#4cc3f1'> 1.1 Mối tương quan giữa thương hiệu đối với giá rượu  </h3>

<h3 style='color:#c177e4'> Bước 1: Lấy ra một số thương hiệu để đánh giá </h3>

* Từ bảng `cat_col_info_df` ta thấy có đến 1620 thương hiệu trong khi dữ liệu chỉ có 5598 dòng. Vì vậy sẽ có nhiều thương hiệu xuất hiện tương đối ít. Ta cần chọn ra các thương hiệu có số lần xuất hiện tương đối cao để có thể đánh giá sự phụ thuộc giữa thương hiệu và giá rượu vang 

In [ ]:
# Tính số lần xuất hiện của mỗi thương hiệu
count_appear = df['Thuong_hieu'].value_counts()
count_appear

In [ ]:
#Lấy ra các thương hiệu xuất hiện từ 25 lần
brand_top = count_appear.index[count_appear >= 25]
brand_top

<h3 style='color:#c177e4'> Bước 2: Tạo dataframe chỉ gồm các thương hiệu đã chọn</h3>

In [ ]:
lis_brand_top = brand_top.values.tolist()
mask = df['Thuong_hieu'].isin(lis_brand_top)
brand_vs_price = df.loc[mask,['Thuong_hieu', 'Gia']]

<h3 style='color:#c177e4'> Bước 3: Trực quan </h3>

In [ ]:
plt.figure(figsize=(18,10))
sns.boxplot(data=brand_vs_price, y='Gia', x = 'Thuong_hieu',showfliers=False)
plt.xticks(rotation=45)
plt.title('Phân bố giá rượu của một số thương hiệu', fontsize=14)
plt.xlabel('Thương hiệu', fontsize=14)
plt.ylabel('Giá rượu (VNĐ)', fontsize=14)
plt.grid(True)
plt.show()

<h3 style='color:#c177e4'> Bước 4: Đánh giá </h3>

🤔🤔🤔 Từ biểu đồ trên ta thấy:
* Hầu hết các thương hiệu được chọn có giá các sản phẩm phân bố nằm chủ yếu trong khoảng từ 200.000 VNĐ đến 1.000.000 VNĐ. Trong đó các thương hiệu `7Colores`, `Vignobles JeanJean` ,`Convha Y Toro` nằm hoàn toàn trong khoảng trên.
* Các thương hiệu `Ampelidae`,`Montgras`, `Tinazzi`, có khoảng từ 25% đến 30% sản phẩm có giá cao hơn các sản phẩm còn lại  thuộc cùng thương hiêu và nằm trong khoảng giá  1.000.000 - 2.000.000 VNĐ.
* Thương hiệu `Borie Manoux` và `Paul Jaboulet Aine` có trên 50% các sản phẩm có giá từ 1.000.000 - 4.800.000 VNĐ.
* Thương hiệu `Louis Jadot` là thương hiệu có nhiều sản phẩm có giá cao và vượt trội so với các thương hiệu khác khi giá cao nhất có thể lên đến khoảng 8.000.000 VNĐ

<h3 style='color:#4cc3f1'> 1.2 Mối tương quan giữa màu sắc đối với giá rượu  </h3>

<h3 style='color:#c177e4'> Bước 1: Chọn những màu sắc xuất hiện nhiều </h3>

* Tương tự với thương hiệu ta sẽ chọn ra những màu sắc nổi bật hơn 

In [ ]:
# Tính số lần xuất hiện của mỗi màu sắc
color_series = df['Mau_sac'].value_counts()
color_series

In [ ]:
# Chọn những màu sắc xuất hiện trên 25 lần
color_top = color_series.index[color_series >= 25]
color_top

* Ta thấy có hai lần ` vàng nhạt ` xuất hiện hai lần. Ta sẽ chuyển chúng cho về giống nhau trong `df` và giữ lại 1 trong `color_top`.

In [ ]:
mask = df['Mau_sac'] == 'vàng nhạt'
df.loc[mask, 'Mau_sac'] = 'vàng nhạt'
# Xóa bớt 1 vàng nhạt
color_top = color_top[color_top != 'vàng nhạt']

<h3 style='color:#c177e4'> Bước 2: Tạo dataframe chỉ gồm những màu sắc đã chọn </h3>

In [ ]:
lis_color_top = color_top.values.tolist()
mask = df['Mau_sac'].isin(lis_color_top)
color_vs_price = df.loc[mask,['Mau_sac', 'Gia']]

<h3 style='color:#c177e4'> Bước 3: Trực quan </h3>

In [ ]:
plt.figure(figsize=(18,10))
sns.boxplot(data=color_vs_price, y='Mau_sac', x = 'Gia',showfliers=False)
# plt.xticks(rotation=45)
plt.title('Phân bố giá rượu theo một số màu sắc', fontsize=14)
plt.xlabel('Giá rượu (VNĐ)', fontsize=14)
plt.ylabel('Màu sắc', fontsize=14)
plt.xticks(np.arange(0, 5000000, 500000))
plt.grid(True)
plt.show()

<h3 style='color:#c177e4'> Bước 4: Đánh giá </h3>

🤔🤔🤔 Từ biểu đồ trên ta thấy:
* Các màu rượu như ` trắng `, ` vàng chanh`, ` hồng nhạt ` có trên 80% các sản phẩm có giá từ 200.000 - 1.000.000 VNĐ.
* Các màu rượu  `vàng nhạt`, ` vàng rơm `, `đỏ ruby` có từ 55% đến 70% các sản phẩm có giá từ 200.000 - 1.000.000 VNĐ. Các sản phẩm còn lại thuộc những màu sắc này phân bố trải dài đến khoảng 2.000.000 VNĐ. Riêng với màu `đỏ ruby` giá cao nhất có thể lên đến 3.000.000 VNĐ
* Màu rượu ` vàng ` có nhiều sản phẩm có giá từ 2.000.000 VNĐ trở lên nhất so với các màu khác. Tiếp đó là các màu ` đỏ `, `đỏ ruby đậm `, `đỏ đậm ` là vượt trội hơn so với các màu còn lại . Trong đó màu ` vàng ` màu có sản phẩm có giá cao nhất tiếp đến là `đỏ`, `đỏ ruby đậm `, `đỏ đậm `.

---